In [1]:
import re
import pandas as pd
import numpy as np
import regex

### Python code to extract Date from chat file 

In [2]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    #pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}' # for New Group settings
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [3]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [4]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [5]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'WhatsApp_Chat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    
    #print(fp.readline())
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

29957


In [6]:
df

,Date,Time,Author,Message
0,2021-05-11,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...
1,2021-05-11,8:33 PM,Celal PUCIT,Karain gey 👍
2,2021-05-11,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,2021-05-11,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,2021-05-11,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶
...,...,...,...,...
29952,2021-08-17,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...
29953,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr
29954,2021-08-17,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂
29955,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂


In [7]:
### Checking shape of dataset.
df.shape

(29957, 4)

In [8]:
### Checking basic information of dataset
#df.info()

In [9]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df = df.replace(r'<Media omitted>$', np.nan , regex=True)
df.shape

df = df[df['Message'].notna()]

In [10]:
### Checking no. of authors of group
df['Author'].nunique()

34

In [11]:
### Checking authors of group
df['Author'].unique()

array(['Ayesha PUCIT', 'Celal PUCIT', 'F.Bilal Uol', 'Umar Ghaffar PUCIT',
       'Amna Ali PUCIT', 'Ayesha Kiran PUCIT', 'Abdullah PUCIT',
       '+92 323 4446586', 'Khurram PUCIT', '+92 322 4155225',
       'Saad PUCIT', 'Adnan Ashraf PUCIT', 'Iqra Aslam PUCIT',
       'Shakir Jamil', 'Muflah PUCIT', 'Arshia CH PUCIT',
       'Usman CH PUCIT', '+92 304 5307578', '+92 300 4611646',
       'Muhammad Faraz PUCIT', '+92 306 2628299', 'Hafsa PUCIT',
       '+92 321 7222267', '+92 307 3900838', 'Aimen PUCIT',
       '+92 303 5018620', 'Nabeel Jutt', '+92 322 6678540',
       'Saad PUCIT MSV', '+92 335 0607000', '+92 335 5027377',
       '+92 306 4093886', 'Aneesa PUCIT', '+92 303 9326488'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [12]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)

### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
0,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...,57,12,0,0
1,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Karain gey 👍,12,3,0,0
2,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta...",51,7,0,0
3,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,46,10,0,0
4,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶,46,11,0,0
...,...,...,...,...,...,...,...,...,...
29952,2021-08-17,Tuesday,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...,77,17,0,0
29953,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr,20,5,0,0
29954,2021-08-17,Tuesday,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂,27,6,0,0
29955,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂,3,1,0,0


In [13]:
### Extracting basic statistics from the dataset
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

Group Chatting Stats : 
Total Number of Messages : 27694
Total Number of Media Messages : 0
Total Number of Links : 0


In [14]:
#Extracting basic statistics of each user : 
l = df.Author.unique()
for i in range(len(l)):
  ### Filtering out messages of particular user
  req_df = df[df["Author"] == l[i]]
  ### req_df will contain messages of only one particular user
  print(f'--> Stats of {l[i]} <-- ')
  ### shape will print number of rows which indirectly means the number of messages
  print('Total Message Sent : ', req_df.shape[0])
  ### Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word\'s']))/req_df.shape[0]
  w_p_m = ("%.3f" % round(words_per_message, 2))  
  print('Average Words per Message : ', w_p_m)
  ### media conists of media messages
  media = sum(req_df["Media_Count"])
  print('Total Media Message Sent : ', media)
  ### links consist of total links
  links = sum(req_df["Url_Count"])   
  print('Total Links Sent : ', links)   
  print()
  print('----------------------------------------------------------n')

--> Stats of Ayesha PUCIT <-- 
Total Message Sent :  1619
Average Words per Message :  5.920
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Celal PUCIT <-- 
Total Message Sent :  617
Average Words per Message :  7.170
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of F.Bilal Uol <-- 
Total Message Sent :  3338
Average Words per Message :  9.190
Total Media Message Sent :  77
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Umar Ghaffar PUCIT <-- 
Total Message Sent :  1649
Average Words per Message :  7.010
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Amna Ali PUCIT <-- 
Total Message Sent :  230
Average Words per Message :  8.260
Total Media Message Sent :  0
Total Links Sent :  0

-------------------------

In [39]:
data = [df['Message']]
rd = pd.concat(data, axis=1)

type(rd)

pandas.core.frame.DataFrame

In [40]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27694 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  27694 non-null  object
dtypes: object(1)
memory usage: 432.7+ KB


In [41]:
### new Data Frame
#rd = df['Message']

### Remove all Emoji and other Symbols
rd = rd.replace(r'\W*$', '' , regex=True)

rd = rd.replace(r'^\s*$', np.nan , regex=True)

#rd = df[df['Message'].notna()]
rd = rd[rd['Message'].notna()]

In [42]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24457 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  24457 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [43]:
rd

,Message
0,Hmme na pagal ho chuki har teacher ki mintaien...
1,Karain gey
2,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,Or phir teachers k krny waly kam b me he kron
...,...
29951,😂😂😂😂 Bus Focus on your study you will get soon
29952,I don't need that type of aurat jo MSCS ki wja...
29953,😂😂😂 Fir to chutti kr
29954,Aho chutti hi ki hoi hai


In [44]:
#rd = rd.sample(frac = 1)

In [46]:
len(rd)

mode = [''] * len(rd)

print(mode[:10])

rd['Mode'] = mode

### Rearranging the columns for better understanding
rd = rd[['Mode','Message']]

rd['Words'] = rd['Message'].apply(lambda s : len(s.split(' ')))

['', '', '', '', '', '', '', '', '', '']


c:\users\hassan raza\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [47]:
rd

,Mode,Message,Words
0,,Hmme na pagal ho chuki har teacher ki mintaien...,12
1,,Karain gey,2
2,,"Kbhe sessional, kbhe assignment kbhe presenta...",7
3,,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,10
4,,Or phir teachers k krny waly kam b me he kron,11
...,...,...,...
29951,,😂😂😂😂 Bus Focus on your study you will get soon,10
29952,,I don't need that type of aurat jo MSCS ki wja...,17
29953,,😂😂😂 Fir to chutti kr,5
29954,,Aho chutti hi ki hoi hai,6


In [23]:
#rd.to_csv('data.csv', index= False)  ## file size 20k+ ......

In [48]:
rd = rd[rd.Words != 1]
rd = rd[rd.Words != 2]

In [49]:
rd

,Mode,Message,Words
0,,Hmme na pagal ho chuki har teacher ki mintaien...,12
2,,"Kbhe sessional, kbhe assignment kbhe presenta...",7
3,,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,10
4,,Or phir teachers k krny waly kam b me he kron,11
8,,Allah kry maan jay ab bus,6
...,...,...,...
29950,,Glt jawab aisi aurtain hmain q ni mili,8
29951,,😂😂😂😂 Bus Focus on your study you will get soon,10
29952,,I don't need that type of aurat jo MSCS ki wja...,17
29953,,😂😂😂 Fir to chutti kr,5


In [50]:
message = rd['Message']

In [51]:
type(message)

pandas.core.series.Series

In [28]:
for i in message:
    print (i)

Ye wesy option achi h saad is m jannat ki guarantee hai
Bhai agr aisy jana pary ga to meri toba
Kia pta sir ka dil hi na kre itna lmba msg prhne ka
Tu mera putt chuti kr
Yar ye likhny m ni ho rha wesy kesy hga
Serious keh rha hon.... Mera tu 2 weeks tk hi final hu ga.... But Meri try hy k elective courses new campus say hi lon...tu is liyay keh rha k m suitable ni hon
1.5kg hai tu
🔥🔥🤣🤣 Noice thinking
Ni ye mera zoq ni
Fir party dy dy
Uni start ho jaye gi jis din uss din sochain gy planing bna lain gy koi
So g AP ja KR yeah shar o shair e AP k bus ki bat nhe hai so jain ap
Haye sadky veer Aya hai
Uoh awr mah veeer
Mid = 15/15,  final = 50/50,  presentation = 0/0......total marks =  65 / 65......iska kia matlab hai koee bta day mei nay abh daikha
Hahhahah.. Aj bilal such me hila hwa
Oh asha me smjha shaid marks ka keh rhe
Yrrr sb ignore kren.. group hi leave kr jaen
Wese suna deti ap.. Aisay sunana laazmi tha kia
Mjje b nai pata
Ye kely se bndy mar deta hau
Hahaha jealous people
Keh kon 

@923204196291 ne mna kr dia hee
G...Teachers me se woi ek hen Jo regularly aty uni
yehe baat tu mje phone pe keh ra tha
😂😂 ooo baaii.. hosla kren.. ab mene kia kaha he ap ko
Oh jany do yar is m knsi bisti
iski class abi leni thi
Rockstar de stickers pej reya
Sir ny first week m hi kafi kaam dy dia h
Hahaha ly lena. Ya b koi baat ha bhala
Ch shb a gaye
Jahan sy easy hua
Qsm see... 😂😂 Pure BS or MSC k qisse sunae the
Aho bus jharo sharo laga dia kri
Movie hi hai na jo ye download kr ra
Wo join kr lo ap bhi
Itni to jisko hni chahiye usko b fikar ni hai
Pm ma send kr
Qsm se😂🤣 glt nikl gye hen
Mujhy bhol hi nai raha
Kisne de diya. Apko to pta b ni ky yaha kya hta
Koi Allah ka bnda ya bndi DM ki sari slides download kr k yaha snd kr skta / skti hai
Paper leak krne ki treat nhi hoti DC call hoti he😆😆😆 @923231488332
Welcome back returns @923496838548
Bhai jwab saaaf saaaf qarara tez mirch masala. Dal ky
‏کبھی تو ختم ہوں گی یہ اداسیاں یہ تنہائیاں اک دن تو اچھا ہو گا چار دن کی زندگی میں
Yaha na 

ye to apki nzr ka Kamal he
Exactly abdulla ko nikalo
Acha dakhty hain
Ye quiz wednesday waly lec m h na
Hahaha Shakir ya kya chuss ha
m bi hoan lekin  ap mujy mujram smjhti
Tu na mery agy baitha kr jesy e neend aye gi ma chpad mar du ga
Han yay best h .. Monday tk wait na karein
Plz even confusion reh jani
bachio ki. trh nakhre q krta e storeys
🤣😂Kia qasooor h piyaar ka.. Jo bandaaam kr rkha ussy tum n mil ke
Hhha han nikal
Jb ID card bna ta
same here me bhi ni krwa sakti... jahan se paisy niklwany wo bank bhi qareeb h or jidhar submit krwani wo bhi but ATM card ni h mere pass
Sai e sai kri jao shaper triple chruble.... Hun ni me krda koi v gal
Kheir itni b bt ni hai ab
Ha na logo ka to kam e ye hai so forget them
comment box me likho
Ye trading hai
To SE me 3 week prhaya ta. Or kuch smj ni ata ta
Chal chup kr ja tu ab agr tujhe pta ha to
Oho jalabi bai.... Naaam jalebi bai
hahahhahaa na Cher malanga nu munafiqat di gal na kr
Ye tw zulm hai
underestimate kar ra mery ko ye
Matlab kya ho

kya mast idea hy rey
Ye shakir ko hi pta o
ap mazak smj rahe i 'm serious
Koi ni aisa moasam ho ga eid k baad bhi
*Mera 18 marks sy A ban rha ha to koi chance ha A hony ka
Kindly koee wo chapter 3 wala document send kr day ya bta day jisko confirm pta ho
Email me yehe likh k 3 elective register krwany hain
Chlo Apki Maan lyty
Mphill k bad
Matlb tujhe umeed ha k asi mil e ni sakti
Iqra batao phr marks ap b models verification vali assignment k
Isi ka dr tha
Ayesha ko bolo session rkh lay kal
is m emoji ka tuk banta
Bai marna tey unj e mar dey
Bs 1 ghantay m aa rahay hain
Wo sary ss dekh lena
Hahahhaha.. Exactly neai ki tlash shuro ho jati ha
Tujh py thuuu tery ustad py lanat
Peer ki zarorat nahi rahe
😂... Yr odey to bghair mza v ni anda
Sir sara semester aisay parhana h
saad wese hi pand hy
Ab phr kya kry. Ya to koi pta karva ly k sir ny Monday ko ana ha k ni
Me ty personally kr rha tha
Wo aa he na jay
Me to group waasiyon ko dekh k kaha
Meri traf sy b done kr
Isi liya se hn
Baaaii ajee

hahah aho.... bhai. senti krna ta sir ko. ab shaid ho jayen
Eda tu pakistani tere nam
Is lea tu mera classmate h
Chalo koi ni
Bilal me tujhy admin bna ra
topper k 91 marks hain aur uskay baad 83 hain
han phr jaan churwana mushkil o jata
matlb kuch b
Ha ma na kal lena hai truck bhar k ada
Ni ni yar in sy koi ikhtlaaf ni wo apny experience ki base py bt kr rhy koi msla ni
Bs woi kr rahe hain abi tk
Adnan shb ap ko Pm m bhaj dyn gy
Nhe nhe to Bhai hai apna
Ya dramy kahi aur ja k kia kr. Hum tujhe janty hn achy sy
Jan wala word bhabi k loye save rakh yaha na use kr
Apko b gussa ata
Thek ho gya
inka hisab lagaty hain
Kabhi teacher keh dy na k shor aa raha hai hm kehty thy peechy se aa raha aur hala k last row hmari hoti thi
Baaaaiiii me jahan khri hoti hun.. team wahan se shuru hoti he.. Acha Sorry
🤣🤣Pata nai biro msg aya me n kaha yahan b keh dn
Tum log itny senti kiun ho jaty jaldi e
4 5 ni 9 10 bolo
Lekin bs ab guzar gya. Ab mera CGPA 3.7 k aas paas hai
Ab sara kuj tery muh pr hi keh rah

...

😶😶😶 uff... Ghalat andaza ho ra
Ustad g Jesy Ap kahy
Han yeh hadsa kb howa Babhi Wala
But mujhe to ha na
Yaar vase es traha to thand pa jandi a na
or aj to samjh wali baat he koi ni idr to
Ha hai saad munafiq phir
mana mera weight ni h zyada per ab iska mtlb k bolon b na
ap single dete raho tasliyan Gem tu Gem h
NLP kaisa subject h kisi ne liya h lecture
Bai vo vali jaan ko baat ni ho rhi
RM credit hours 3??? ae ki mazak ay
First welcome sticker
Yay apkay leyay bhe h 😎😂 ab nikalein yahan se after Mphil
ap challan form bhi old campus me hi submit krwain gi na
Dash na dash ko dash se dash mara
Me *RM waly group* me aik msg bhjne lga ho sab usko respond kren.  *apne name or roll.o mention kr den* us msg ky reply me  *jinki RM ki presentation rehti
Requesty to boht kr li hn. Ab bus pao pakrny valy ra gy hn sir k
Wese mene abhi tak ni socha kuj
Bai gussa kio krna yara tey
Leave kr ja
Yar phir b sb k samny esy be izzat hna
Ya mamoo shakir. Which one is bst
Other iz girlz
Han wohe na...inka ko

kia khyal e
Abi nhi kal se
Me ap ki support me hun.. tntion na len✌🏻😂 #SupportAbdullah
Abi result final nai hua
To btien naw
😂 bai me sab se best candidate hu
Hmari cr bht masooom h.. Ussay aisi complex baatein smjh nai ati
I got u
Dakh bhai ma tuj tk abi ni pohncha
Yar msla bhla kya h koi bt sunna ni chahta yahan
Mery sath b yahi hota tha
Ha to woi important hai aur kia hona bhala aj
Umar bta skta hau ye to
g bro Allhmadullilah
Chal ab ja b
Dekhin kuch reh na jayein neechay
Keh kon raha hai kamina
Ustaad g. Mein bhi hu binance pe 😂 mujhay sikha rahay ap
Sady nal raho gy tu new new techniques pta chlti rhy gi
Uni ja ky sakun da saa aya aj
Welcome back chutki.... Aj to baad tusi group left ni krna... Group di jaan tey shaan o tusi(2
Na lain.... first me second krwa dain
rehny dy m ab asi batien ni sunta m sharef insan hoan
Raat nu 45 minutes hi duta si me fer uth gya. Odo baad ni ayi neend
Un logo ne uper se taap kiya humne nechy se
😂 bai me tenu janda e ni to hai kon
Ayesha lagta ha ap

Kheir koi ni 1 din to ye hna e tha
lekin asy thanks ki koi value ni bnti
Hun gaib ho gya e
Ni abhi ni aya
never we are sincere to eachother
Lahore me Garbage kitna collect hota per day.. Zara cost estimate krna
Haha apayi yes krao usne black vigo bhji h
Itni jaldi b kia h soney ki.. Abhi 3 e to bjy hn
Ro ro k jb insan thak jata h phir hasna e prta h
Kehty ho to yahi bta deta hu bs rona ni hai
Mjy to bs ek he gazal yad hai.. Usky bad kbhe koshish he nh ki
*Chaudhary Muhammad Saad bin Shahbaz Ahmed Virk
Abdullah meri jaan hy usey kisi ne kuj ni kehna or ye remove krney wala mazak sirf me kr sakta usky sath
change kro adat
Ak presentation pr 40 mil gay hn.... Is hisab sy socha jay to theak hi h
its up to u
gb whatsapp ki jahan zarort pary bahi ko bata dna
Ya b ha
Hmary to ye scene hota tha time se ja k bhi end py e baithna hota tha
😂😂😂 Len g jigars ki baat dil pr le gye abdullah shb
Pehli dfa izzat hui e
Farig to ma e hota hu na
This is business
apun ni kehta
Aur wo bhi kafi doongi
Apas m

This message was deleted
Tusi ty great ho
Har mazak ko cheez banaya huva ha
Ab hum lrko ka b group bnayein ge
Ma Muhammad Bilal Abid Sandhu keh rha hu
Yr 1 credit hour ka hai mera ni khyal zyada frq pry ga
Pura 1 minte zaya hua h mera
Adnan ya acting tu ny b waha sy e to ni sekhi jaha sy jannat mirza ny sekhi thi @923420431383
Ap lagta ha chari sy dar gai hn
Inni v gal ni ay
idr wo wala scene ni h idr boht agy ka scene h
Hm kuch bara soxhty hain.... Is liye... Hm soraj pe ja rhy hain
sahi kaha lekin final ki ratio 50 me se nikali thi...as u know paper 45 ka tha
Ye to saad lgra he
Bta phir kb choro group
Zalim hukmran hain... Mazlum awam pe zulm krty hai
original version bta kasa tha
Bada shoq hai tujhe adnan ko personal ma dakhny ka
Vo khud b leave kr gai ha. Ankhy khol k dakh
m wese ni hoan yr jasa tum smjhty ho
Hospital ka name
Hm na to ni kia
Bhai ose sara ata he
Mazak ey yr
Mjhy ni manage krna
Ha yar hmain bhol gai
Aur ma wo qatal tera krta
@923484608364 uni a gya ha
random number 

🙄 @923054524120 smjhe lein
Kuch nhe hota
Run mureedi ki
@923124533514 apki zrort hai idhr
Ap sab dua krna please
CR is baat pe CRship le leni chaye apse
Hahahahahhahah isey smjo na isey enjoy kro
Tje koi ni manaye GA.... Tu beshak na aye
Mera b plan ha k InshaAllah PhD iub sy e karni ha
Ye ni itna azzall hai😂😂😂😂😂 ye ni prta thnda
Alright then thik hai
Ahoo. Hun sab di entry bnd ais group cho
Wah wah wah adaab adaaab
11.30 pa b thi na koi
Game time busy ni hoty na
Ur good name plz
Abdullah to kabab ma haddi hai na
@923054524120 dude shukriya
abhi presentation k lgny do
Ap iqal ko e prhain phir
tum kon ho
Kal roza h
Haha aho kehnda me CR no
Haha bs*k group me me akela usman ky sath lga ta ajata to v
Nai nai.. Fr abhi ap n lagta aisi bryani khai nai... Iska taste ufffff kia btayein ab hm
Ye ni btaye ga wo
Sahi smjh rhi ho iqra
Or kisi ki himmat ni oti k wo ghar baat kr ske
Whi kah ra k kafi logo ny bt ignore kr di thi.... Ya unhain pta nai chla.... Ap ny unka b dhyan ais trf kr diya
Hahah

In [29]:
import re
reg = '[^\sA-Za-z]'
s = 'Sir bus hamri thori izat rakh rhy hn aur methi methi kr rhy hn'
s= '😂😂😂😂😂😂😂 error to me beech me se nikal lu ga oska fiker na kro'
rs = re.findall(reg,s)

In [30]:
len(rs)

7

In [52]:
def remove_emoji_msg(msg):
    import re
    reg_Exp = '[^\sA-Za-z]'
    len(msg)
    new_msg = []
    for i in msg:
        rs = re.findall(reg_Exp, i)
        if len(rs) == 0:
            new_msg.append(i)
    return new_msg

In [53]:
_list = remove_emoji_msg(message)

In [54]:
_list[:10]

['Hmme na pagal ho chuki har teacher ki mintaien kr kr ky',
 'Or phir teachers k krny waly kam b me he kron',
 'Allah kry maan jay ab bus',
 'Abhe aca ki assignments rehti hain',
 'abi tk next koi nhi di tw abi nhi rehti',
 'Ab Eid tu guzarny dein sakoon se',
 'Eid valy din b hum assignment e kry gy',
 'Allah na karay',
 'Lagta ha sir kisi aur ka gussa hum pa nikal rhy hn',
 'Samjh samjh ki bt hoti']

In [55]:
len(_list)

14282

In [35]:
print(type(rd))

for index, row in rd.iterrows():
    print (row["Message"], row["Words"])

<class 'pandas.core.frame.DataFrame'>
Ye wesy option achi h saad is m jannat ki guarantee hai 12
Bhai agr aisy jana pary ga to meri toba 9
Kia pta sir ka dil hi na kre itna lmba msg prhne ka 13
Tu mera putt chuti kr 5
Yar ye likhny m ni ho rha wesy kesy hga 10
Serious keh rha hon.... Mera tu 2 weeks tk hi final hu ga.... But Meri try hy k elective courses new campus say hi lon...tu is liyay keh rha k m suitable ni hon 34
1.5kg hai tu 3
🔥🔥🤣🤣 Noice thinking 3
Ni ye mera zoq ni 5
Fir party dy dy 4
Uni start ho jaye gi jis din uss din sochain gy planing bna lain gy koi 16
So g AP ja KR yeah shar o shair e AP k bus ki bat nhe hai so jain ap 20
Haye sadky veer Aya hai 5
Uoh awr mah veeer 4
Mid = 15/15,  final = 50/50,  presentation = 0/0......total marks =  65 / 65......iska kia matlab hai koee bta day mei nay abh daikha 27
Hahhahah.. Aj bilal such me hila hwa 7
Oh asha me smjha shaid marks ka keh rhe 9
Yrrr sb ignore kren.. group hi leave kr jaen 9
Wese suna deti ap.. Aisay sunana laazmi th

Yar sir to chill hua krty thy 2 subj prhy huye m ny in sy 15
kis cheez me 3
Esy kisi or ko mnaty to situation kuch or e hti 11
Ab esy bolo gy 4
Oh.... Me darda nai 4
Ab utha hy apun 4
Lagda menu apni power wakhani paini aik wari fer 9
Bady seedy andazy hain apky 5
Ohh hn. Ma b karta hu dusry number pa gb aur vo number yaha add kri 16
Haha bai me Desi ghee lgata.... Makhan ni 8
i was just waiting for this 6
to whom  it may concern 6
Celal qasm sy tu heera hy bai 7
Yr 2-4 din ka time lo. Ye log pagal h kya 11
55 hen abi 3
3 k sath bhi or 3 k baghair bhi 9
Yaar mera bara dil kr rha k Teri class lu vaha pa. But bhr k log hn vaha 18
Tmhara to Dana pani hi wahan pe h 8
Yhi faida hota CRship ka... Interaction ho jati sb k sth 11
Abdullah k doston ko kuch  nai khna bs 9
m ny kahakyah 3
What's going on 3
Aur vele rehny k liye tarsy howy log 8
tum log apas m khush hoty raho 7
Cuteness kitna faida deti yr. Kamaal hai wesay. If you're cute, ap azaad ho 14
To us m b bs bhabi ko e dikhayen na 🙊😂 ya 

Inhain remove kr rhy hain ya ni 7
Na kaho Saad block krde ga 6
Vo to hai bai 4
🤯🤯🤯tu ne galat likha ya mjhy galat nazar aa ra 10
Roll number 5 ki mail aa rae hain 8
Yeich chz chaye 3
Chal sai h 3
Kon mila hai 3
Bai paisa bnany ki scheeme achi h 7
Te hun khedna shuru kr de. Rockstar 7
Ya hum logo ko nikal dy ga 7
Ab jana ni hai 4
Isi liye jldi out ho jate hen 7
Apki presentation nhe huee thi 5
Apko block Kia hy to q Kia apko b pta hy 11
This message was deleted 4
Tujhe to personal ma bhi ani hain 7
This message was deleted 4
Saad aggo chup ay 4
Kon hain ye log 4
No doubt he is 4
ابھی تو راکھ ہوئے ہیں تیرے فراق میں ہم ابھی ہمارے بکھرنے کا کھیل باقی ہے 16
Try kro na 3
Ayesha ne previous msg copy kr k paste kia.. may be she forgot 13
Abhi thori der pehle aimen ka phone gira he... Dobara girana he 12
😥😥😥 emotional q ho Ra....kafi girls yahan ni to me bs is lye Soch Rae thi hmari girls ki b dosti honi chahye na so we need separate room 29
Chuti is so weird. Koi acha naam hona chaheay 9
Is gr

Hlka sa pighli hain yar 5
G g smjh gai 4
Wah bai. Chal enjoy kr 5
@923484608364 plzzz..... Aisy b 4
Ya kya mazak ha yaar 5
Sub boys ko mention kiya hwa.. Including you 8
Khbardar ab chawal mari kisi ny 6
Had h ye gemini wala rona ni mukta iska 9
Link send kro 3
I prefer on site classes I hate online studies 9
haha umr kolon ko pocho @923054524120  eenu das 9
is group m kisi ka ni toota 7
Lollypop se kon sad hota hai 6
Ap pehl kren me cheers krun gi ap ko.. Shaaaakkiirrrr shaakir huu huuu 13
Tu ty chup kr 4
Haha na baii mujhy kya zrort hai k ye kron 10
yeh seesha ni dekhta 4
Happy birthday veery 3
Wo lfz agy pechy ho gaye thy 7
Kon ara kon ja ra 5
Allah esi b kya biryani ki muhbt 7
kisko masla h 3
Collection bnai jao ap😴😴kya yad kro gy ek zamany me ek bht e Achi bandi aai thi 17
@923420431383 bro jokes apart apki acting achi he ap movies me try kro 13
ahahhahahahah IN SHA ALLAH 4
Aur bhabi ko bhool gya tu ye draz walo k chakar ma 12
Ni bhai Ni kabhe bhe na 6
Data mining Ka lectr Thursda

Aho bata du ga vahi e 6
Serious e ho gaye ap tw 6
Sir b thandi jugtien he lgaty hain 7
Chalu sb sir ko thank you bolo 7
Haye... Yeh ha mera veera 5
yr tu mera pecha na chadi 6
Fer bna lisht 3
Mein nay kaha tha NCAI wali larkio sey dosti kr lo. Apkay bhi chokay aa jain gey 17
No issue yrr 🥰.... we should wait for official list... otherwise ap best ho 14
Km lg rahe hain mujhe to 6
keh kon raha e 4
Phi add kro dobara usy 5
Ha to wo bant skta hai 6
Tu hans q raha hai 5
Sad 😕 wait kro phr 1 bjy tk 8
Bhai ab tu jan bojh k bt ko uss trf ly k ja ra 14
Apko meri video nazar aa rhi kia 7
G lekin her koi abhi ni krwa skta na msla ye h 12
Woi kam krana kia 4
😅😅😅bai aram nal chand mari. Meri seht weekh tey apni weekh 11
Sahi he phr.. bilkul sahi ja rhe hen 8
Ha yaha ma interested hu 5
Yaar adnan hero lag rha ha vase South Indian movies ka 11
Hahaha sir khud sy e batein kr rhy thy 9
Barish ki wja sy complete ni huva. No results 9
🤣🤣🤣Ab ye prh k mjhe v kuch yaad a rha. @923106905056? Do you 13
ye hi 

...

mje sir ki awaz clear ni a rae 8
اکثر لگتا ہے بیکار چاہتا ہے یہ دل جو تجھے بار بار چاہتا ہے  تو اگر دفتر میں ساتھ کام کرے کون کم بخت اتوار چاہتا ہے 28
This message was deleted 4
Knsa fine koi fine ni hta 6
🔥🔥😂😂 bilkul bja frmaya ap ne 6
itni bari ghalati 3
Yar link send kri class ka 6
Welcome back shakir shb 4
😂😂🤣🤣 ap to ese keh rhe jese ACE kraya ho ap ne mje.. or mene shukria nhi kaha 18
Iska bht mana giraaami jawab hai seriously 😂😂pr keh nai skta koi 12
beard k bgair saad 4
Abdullah to krta rehta hai 5
Mery khud plans ki asi ki tasi hui parhi ha 10
🙄🙄 @923054524120 smjha len inhe 5
Ye bhai behan wala mazk kahan sy ajata h yr 10
Han chal gya tha pta mujhe 6
Tu IuB ka ho gya h na ab to aise hi kahy ga 13
Han but ma karta hu dusry number pa aj e 10
*Yr Adnan apko kaisa feel ho rha itna popular ho ky.... Pura group janta hai apko... Sab apki tareef krty nahi thakty...*  How are you feeling today 28
sadqy waari jaon wesy tm logon k pyar k 9
cr ko add kro bas 5
Hahaha Alhamdulillah... Khu

Hahaha ye msg mainy 5 mint phly ka send kiya hwa 11
Dialogue hai yar 3
Han ye b keh skty ho ap.. Ban jati hn 10
Math phly b ap bs sir k mzak pr hs hi rhy thy 13
Choky mari ja rahi hain aur kehti sawad e ni aya 11
Chawww krny aye thy...ja Rae haseen khawabon ki waadion me 10
Ak dosry ka jotha kha kha kr 7
Bryani ka venue 3
Phupo na bnien 3
Dua ma kya mangna ha 5
Kesi ashleel guftgu ho rhi hai 6
Hahaha bai usne khud e kaha 6
Hahahahha saad Love he tu.... Tu mind ni krta Wrna kuch log hote apna personal gussa group me a kr nikalte 21
@923484608364 @923054524120 daikh yr tery bhai ko chor bna diya 10
yeh kasi uni hy jo eid waly din b ni chorty 11
Love ho gya bro 4
Me keh rhi thii k chor den.. ap ne kaha nhi smjhana he aj b inhe😂😂😂😂 len smjha len ab 20
Aj bholana hi acha hai 5
O Teri khair. Ap ko bari information ha groups ki 10
Ye b theek he 4
Or second me pata nai kia banta 7
unki galia kesi hon ge 5
Mujhy pata tha k ye June ka h 8
🤨🤨Lagta msg pe ghor nai kia Meku ex nai chahie 10
Yaar vo

Ye to woi bat ho gai jesy waqar zaka kehta hai k mujhe Prime Minister bnao 16
Yr esko muh na lagaya kro😂😂😂 shoka ho jata ye 10
😂😂😂😂 aa ty gya hun nazar 6
Ma samjh gya hu vase e 6
Tenu kehn deya 3
Chlo sahii h m kuch bolon to msla ho jata wesy b 12
Tu bhi bdal gya 4
Ye to ho ga baii uss lrki ki dp b to dakho 12
Ab ya na kahi k en ma sy ik tu b ha 12
khawab dekhny p konsa pabandi h 6
Aag ni lgani kia tu na 6
Amna madam q gai hain??? Un k mizaaj ko ab kn si baat buri LG gai 16
Saad apne to ni jana 5
m ny kaha kuch bi ni as such jitna in bewaqofon ny suspense creat kr diya h 17
😂 to v soch 4
ap taal skti hain 4
Han Friday tk ata hu phr 6
Hahaha mujhe b yahi lag rha ha 7
Subha subha mood khrb na kr 6
Mery sath ye fight krien gy to harr jaien gy 10
Ayesha u r the best dude....ap e rho 8
Already ACA meray dimaagh say abh b nhe gya 9
Ap b rakh lain 1 sath eid krein gy 9
Indirectly kisko sunana chah rha 5
Iss se na ho paye ga 6
Bilal ghor kr 3
Ho rhi fr shuru ch shb 6
chawla jai sai das 4
Sary 

Haha or sunayen sy agy kuch smjh e ni ati 10
dream job sird dream m hi hoti 7
Bat khatam hogae h 4
Ap dekh lena... Agar ap ko lgta howa k ap thesis k sath handle kr sakti ho... Then go for it 21
Han na... Janey den ungy 5
m q aaon tu aja 5
Phly apna to Kar lo 5
Ma b aur baki over time class ma laga leta hu 11
Q din din khrab krna unka bhi 7
tum logo saro ty maro m ny ni share krna tum log 2 numberiyan mere sath krty ho 18
oo ja koi ni kr sakhta 6
Sai ho gya bro 4
Hahaha try na manany sy kya hota ha 8
Tu isnan ban ja 4
Mention page pls 3
Mei nay kal kaha tha mujy highlight na kiya krain apbi batein krain apko meri tension laynay ke zrurat nhe jo mujy tnngg kray ga mei uss say khud nimat lu gi...got it 33
Seriously banda khud sochy. Agr koi hamry sath asa mazak kry tooo 12
Ap k sath esa ho to ni gya khein 9
tu phir mia parh lo 5
krty hain kuch mara time chl raaa 7
Hahaha bai apna dasi... Mera te awain to bna dita 10
aj kam az kam bachon ko pareshani to na hoti 10
Wo jaldi emotional ho ja

KeyboardInterrupt: 

In [56]:
len(rd)

20397

In [57]:
new_df = pd.DataFrame(_list, columns=['Message'])

In [58]:
mode = [''] * len(new_df)

print(mode[:10])

new_df['Mode'] = mode

### Rearranging the columns for better understanding
new_df = new_df[['Mode','Message']]

['', '', '', '', '', '', '', '', '', '']


In [59]:
new_df

,Mode,Message
0,,Hmme na pagal ho chuki har teacher ki mintaien...
1,,Or phir teachers k krny waly kam b me he kron
2,,Allah kry maan jay ab bus
3,,Abhe aca ki assignments rehti hain
4,,abi tk next koi nhi di tw abi nhi rehti
...,...,...
14277,,Or me ne uni a k msg parha h
14278,,Apka dukh alg h
14279,,Mainy isi liya rat he msg kr diya tha k subha ...
14280,,Glt jawab aisi aurtain hmain q ni mili


In [ ]:
# Export dataframe to CSV
#new_df.to_csv('chat_data.csv', index=False)

In [60]:
# Export dataframe to CSV
new_df.to_csv('Whatsapp_chat_Data.csv', index=False)